In [32]:
from sklearn.datasets import load_digits
import numpy as np


def get_onehot(y):
    
    n_classes = len(np.unique(y))
    n_samples = len(y)
    
    y_onehot = np.zeros([n_samples, n_classes])
    for i, val in enumerate(y):
        y_onehot[i, val] = 1
    return y_onehot


digits = load_digits(n_class=6)

X, y = digits.data, digits.target
y_onehot = get_onehot(y)

n_samples, n_features = X.shape
n_classes = len(np.unique(y))

In [33]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=10, ncols=_hid0, figsize=(6, 6))
for idx, ax in enumerate(axs.ravel()):
    ax.imshow(X[idx].reshape((8, 8)), cmap=plt.cm.binary)
    ax.axis("off")
_ = fig.show

NameError: name '_hid0' is not defined

In [ ]:
rand_seed = 28
rnd = np.random.RandomState(rand_seed)

# Hidden layer
n_node_h = 50
weight_h = rnd.uniform(0, _hid, [n_node_h, n_features])
bias_h = 1

n_node_out = n_classes
weight_out = rnd.random([n_node_out, n_node_h])
bias_out = 1

## Feed Forward

In [ ]:
z_h = X.dot(weight_h.T) + bias_h
z_h.shape

(1083, 50)

In [ ]:
a_h = 1 / (1 + np.exp(-z_h))
a_h.shape

(1083, 50)

In [ ]:
z_out = a_h.dot(weight_out.T) + bias_out
a_out = 1 / (1 + np.exp(-z_out))
a_out.shape

(1083, 6)

## Backpropagation

In [ ]:
logloss_out = (y_onehot * np.log(a_out)) + ((1 - y_onehot) * np.log(1 - a_out))
avg_logloss_out = np.average(logloss_out, axis=0)

In [ ]:
new_weight_out = np.average(avg_logloss_out * a_out, axis=0)
new_bias_out = np.average(avg_logloss_out * bias_out, axis=0)

In [ ]:
weight_out = weight_out - new_weight_out.reshape([6,1])
bias_out = bias_out - new_bias_out

In [52]:
class NeuralNetwork:
    
    def __init__(self, X, y, n_node_hid, learning_rate=0.1) -> None:
        rand_seed = 28
        self.rnd = np.random.RandomState(rand_seed)
        
        self.n_node_hid = n_node_hid
        self.n_samples, self.n_features  = X.shape
        self.n_classes = len(np.unique(y))
        self.n_node_out = n_classes
        self.learning_rate = learning_rate
        
        # Initialize weights and biases with random values
        self.W_hid = rnd.uniform(0, 1, [n_features, n_node_hid])
        self.b_hid = np.zeros((1, n_node_hid))

        self.W_out = rnd.random([n_node_hid, n_node_out])
        self.b_out = np.zeros((1, n_node_out))
        
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))


    def sigmoid_derivative(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))

        
    def feedforward(self, X):
        
        self.z_hid = np.dot(X, self.W_hid) + self.b_hid #[1xxx, 64][64, 50] -> [1000, 50]
        self.a_hid = self.sigmoid(self.z_hid)
        
        self.z_out = np.dot(self.a_hid, self.W_out) + self.b_out
        self.a_out = self.sigmoid(self.z_out)
        
        return self.a_out


    def backpropagation(self, X, y):
        # Calculate error and gradients
        delta_out = (self.a_out - y) * self.sigmoid_derivative(self.z_out)
        dW_out = np.dot(self.a_hid.T, delta_out)
        db_out = np.sum(delta_out, axis=0, keepdims=True)
        
        delta_hid = np.dot(delta_out, self.W_out.T) * self.sigmoid_derivative(self.z_hid)
        dW_hid = np.dot(X.T, delta_hid)
        db_hid = np.sum(delta_hid, axis=0, keepdims=True)

        # Update weights and biases
        self.W_out -= self.learning_rate * dW_out
        self.b_out -= self.learning_rate * db_out
        self.W_hid -= self.learning_rate * dW_hid
        self.b_hid -= self.learning_rate * db_hid
    

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.feedforward(X)
            self.backpropagation(X, y)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}: Loss = {self.calculate_loss(X, y)}")


    def calculate_loss(self, X, y):
        return np.mean(np.square(self.feedforward(X) - y))

In [51]:
# Example usage
learning_rate = 0.1
epochs = 1000

# Create a neural network
nn = NeuralNetwork(X, y, 50, learning_rate)

# Train the neural network
nn.train(X, y, epochs)

# Make predictions
predictions = nn.feedforward(X)
print("Predictions:", predictions)

ValueError: operands could not be broadcast together with shapes (1083,6) (1083,) 